In [ ]:
file=open("article.txt","r")
text=file.read()
#print(text)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('popular')

In [ ]:
import itertools
import pke
from nltk.corpus import stopwords
import string
#print("fin")

In [ ]:
def getImportantWords(art):
    extractor=pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=art,language='en')
    pos={'PROPN'}
    stops=list(string.punctuation)
    stops+=['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stops+=stopwords.words('english')
    extractor.candidate_selection(pos=pos,stoplist=stops)
    extractor.candidate_weighting()
    result=[]
    ex=extractor.get_n_best(n=10)
    for each in ex:
        result.append(each[0])
    return result
impWords=getImportantWords(text)
print(impWords)

In [ ]:
from nltk.tokenize import sent_tokenize
def splitTextToSents(art):
    s=[sent_tokenize(art)]
    s=[y for x in s for y in x]
    s=[sent.strip() for sent in s if len(sent)>15]
    return s
sents=splitTextToSents(text)
#print(sents)

In [ ]:
from flashtext import KeywordProcessor
def mapSents(impWords,sents):
    processor=KeywordProcessor()
    keySents={}
    for word in impWords:
        keySents[word]=[]
        processor.add_keyword(word)
    for sent in sents:
        found=processor.extract_keywords(sent)
        for each in found:
            keySents[each].append(sent)
    for key in keySents.keys():
        temp=keySents[key]
        temp=sorted(temp,key=len,reverse=True)
        keySents[key]=temp
    return keySents
mappedSents=mapSents(impWords,sents)
#print(mappedSents)

In [ ]:
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
def getWordSense(sent,word):
    word=word.lower()
    if len(word.split())>0:
        word=word.replace(" ","_")
    synsets=wn.synsets(word,'n')
    if synsets:
        wup=max_similarity(sent,word,'wup',pos='n')
        adapted_lesk_output = adapted_lesk(sent, word, pos='n')
        lowest_index=min(synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None
print("fin")

In [ ]:
def getDistractors(syn,word):
    dists=[]
    word=word.lower()
    actword=word
    if len(word.split())>0:
        word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym)==0:
        return dists
    for each in hypernym[0].hyponyms():
        name=each.lemmas()[0].name()
        if(name==actword):
            continue
        name=name.replace("_"," ")
        name=" ".join(w.capitalize() for w in name.split())
        if name is not None and name not in dists:
            dists.append(name)
    return dists
print("fin")

In [ ]:
import requests
import json
def getDistractors2(word):
    word=word.lower()
    actword=word
    if len(word.split())>0:
        word=word.replace(" ","_")
    dists=[]
    url= "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj=requests.get(url).json()
    for edge in obj['edges']:
        link=edge['end']['term']
        url2="http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2=requests.get(url2).json()
        for edge in obj2['edges']:
            word2=edge['start']['label']
            if word2 not in dists and actword.lower() not in word2.lower():
                dists.append(word2)
    return dists


In [ ]:
mappedDists={}
for each in mappedSents:
    wordsense=getWordSense(mappedSents[each][0],each)
    if wordsense:
        dists=getDistractors(wordsense,each)
        if len(dists)==0:
            dists=getDistractors2(each)
        if len(dists)!=0:
            mappedDists[each]=dists
    else:
        dists=getDistractors2(each)
        if len(dists)>0:
            mappedDists[each]=dists
#print(mappedDists)

In [ ]:
print("**************************************        Multiple Choice Questions        *******************************")
print()
import re
iterator = 1
for each in mappedDists:
    sent=mappedSents[each][0]
    p=re.compile(each,re.IGNORECASE)
    op=p.sub("________",sent)
    print("Question %s-> "%(iterator),op)
    options=[each.capitalize()]+mappedDists[each]
    options=options[:4]
    opts=['a','b','c','d']
    random.shuffle(options)
    for i,ch in enumerate(options):
        print("\t",opts[i],") ", ch)
    print()
    iterator+=1